In [1]:
import numpy as np
import pandas as pd

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_75490/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [69]:
import requests
base_url = "http://localhost:7999"
upstox_url = "https://api.upstox.com/v2"

### NSE Tickers Wrangling

In [29]:
instruments = pd.read_csv("data/upstox/NSE.csv")
instruments.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX


In [45]:
equities = instruments.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [46]:
indexes = instruments.query('instrument_type == "INDEX" and exchange == "NSE_INDEX"')
indexes.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX


GOAL ~ 2400 NSE listed companies
Remove:
- Government issued instruments
- Anything with %
- NSETEST

In [50]:
randoms = ["%", "NSETEST"]
goi_issues = ["GOI STRIPS", "GOI TBILL", "GOI LOAN"]

def remove_names(name):
    return all(i not in name for i in randoms + goi_issues)

equities = equities[equities["name"].apply(lambda x: remove_names(x))]
equities["exchange"] = equities["exchange"].str.replace("NSE_EQ", "NSE")

equities.shape


(2894, 12)

Remove Indexes with no exchange token

In [66]:
indexes.replace(np.nan, None, inplace=True)
indexes.dropna(subset=["exchange_token"], inplace=True)
indexes["exchange"] = indexes["exchange"].str.replace("NSE_INDEX", "NSE")

indexes.shape

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_75490/3341430381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indexes.replace(np.nan, None, inplace=True)
/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_75490/3341430381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indexes.dropna(subset=["exchange_token"], inplace=True)
/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_75490/3341430381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

(43, 12)

### POST Equities to BE

In [59]:
rename_cols = {
    "tradingsymbol": "ticker",
    "instrument_key": "upstox_instrument_key"
}
cols = ["exchange", "exchange_token", "name", "ticker", "instrument_type", "upstox_instrument_key", "lot_size"]


In [60]:
def post_tickers(tickers):
    s, f = 0, 0
    failed = []
    for p in tickers:
        resp = requests.post(base_url + "/tickers/", json=p)
        if resp.status_code == 201:
            s += 1
        else:
            f += 1
            failed.append(p)
            print(resp.json())
    return s, f, failed

In [62]:
equities.rename(columns=rename_cols, inplace=True)
payload = equities[cols].to_dict(orient="records")

s, f, failed = post_tickers(payload)

print(f"Success: {s}, Failure: {f}")
print(f"Failed {failed}")


Success: 2894, Failure: 0
Failed []


In [ ]:
# indexes.rename(columns=rename_cols, inplace=True)
# payload = indexes[cols].to_dict(orient="records")

# s, f, failed = post_tickers(payload[0:5])

# print(f"Success: {s}, Failure: {f}")
# print(f"Failed {failed}")

04.05 - 43 Indexes + 6563 equities posted

05.05 - Deleted and posted only relevant equities 2894 (indexes have overlapping exchange tokens, do NOT post yet)

### Fetch candle stick data and POST to Equities

In [71]:
equities.rename(columns=rename_cols, inplace=True)
payload = equities[cols].to_dict(orient="records")


for equity in payload[0:1]:
    instrument_key = equity["upstox_instrument_key"]
    resp = requests.get(upstox_url + f"/historical-candle/{instrument_key}/day/2024-01-05/2024-01-01")
    print(resp.json())

{'status': 'success', 'data': {'candles': [['2024-01-05T00:00:00+05:30', 175, 178.65, 173.5, 175.25, 92831, 0], ['2024-01-04T00:00:00+05:30', 180, 183.2, 174.85, 175.7, 149833, 0], ['2024-01-03T00:00:00+05:30', 171.15, 180.8, 169.5, 179.25, 261772, 0], ['2024-01-02T00:00:00+05:30', 176.5, 177.7, 168.5, 171, 321759, 0], ['2024-01-01T00:00:00+05:30', 177.2, 182.95, 176, 176.5, 117885, 0]]}}


## External API

In [ ]:
import requests

url = "https://api.upstox.com/v2/historical-candle/NSE_EQ|IN002023Y458/day/2024-01-05/2024-01-01"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)